In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
from pathlib import Path
from numpy.random import RandomState

_PRNG = RandomState(0)

In [2]:
_DATA_DIR = "./data"

dataset = []

for file_name in os.listdir(_DATA_DIR):
    if file_name.endswith(".csv"):
        full_file_path = _DATA_DIR + "/" + file_name
        print(full_file_path)
        dataset.append((file_name, pd.read_csv(full_file_path)))

./data/LoanStats_2016Q1.csv


C:\Users\juszhan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


./data/LoanStats_2016Q2.csv
./data/LoanStats_2016Q3.csv


C:\Users\juszhan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


./data/LoanStats_2016Q4.csv
./data/LoanStats_2017Q1.csv


C:\Users\juszhan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (112) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


./data/LoanStats_2017Q2.csv
./data/LoanStats_2017Q3.csv
./data/LoanStats_2017Q4.csv
./data/LoanStats_2018Q1.csv


C:\Users\juszhan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (123,124,125,128,129,130,133,139,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


./data/LoanStats_2018Q2.csv
./data/LoanStats_2018Q3.csv
./data/LoanStats_2018Q4.csv


In [3]:
for name, df in dataset:
    print(name)
    print(df["loan_status"].value_counts())

LoanStats_2016Q1.csv
Fully Paid            89143
Charged Off           22790
Current               20336
Late (31-120 days)     1032
In Grace Period         409
Late (16-30 days)       176
Default                   1
Name: loan_status, dtype: int64
LoanStats_2016Q2.csv
Fully Paid            46994
Current               33391
Charged Off           15952
Late (31-120 days)      902
In Grace Period         384
Late (16-30 days)       228
Default                   3
Name: loan_status, dtype: int64
LoanStats_2016Q3.csv
Fully Paid            45085
Current               36389
Charged Off           15806
Late (31-120 days)     1183
In Grace Period         442
Late (16-30 days)       215
Name: loan_status, dtype: int64
LoanStats_2016Q4.csv
Current               43945
Fully Paid            43631
Charged Off           13694
Late (31-120 days)     1429
In Grace Period         558
Late (16-30 days)       283
Default                   6
Name: loan_status, dtype: int64
LoanStats_2017Q1.csv
Current    